In [1]:
import tensorflow as tf
import gudhi as gd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utils
import scipy.spatial as spatial
import json
from rich import print
from random import choice, sample
from tqdm.notebook import tqdm
from rdkit import Chem
from rdkit.Chem import Descriptors
from gc import collect
from pickle import load

with open("9701_cech_persistence_entropy_012.pickle", mode = "rb") as data:
    save_dict = load(data)
    train_molecules = save_dict["train_molecules"]
    train_data = save_dict["train_data"]
    
    test_molecules = save_dict["test_molecules"]
    test_data = save_dict["test_data"]
    
%matplotlib inline
del save_dict
collect()

4

In [2]:
print(train_data.shape, test_data.shape)

(7761, 9)
(1940, 9)

# Extract descriptors
(this can be done from rdkit or from the descriptor .csv)

In [3]:
df = pd.read_csv("data/Drugbank_some_descriptors.csv")
cntr = 0
for d in df:
    cntr += 1
print(cntr)

126

### Filter descriptors which are unavailable

In [4]:
descriptors = list()

#---- consider only descriptors which have few missing values and which are floating point
for d in df:
    if df[d].isna().sum() < 800 and df[d].dtype == np.float64:
        descriptors.append(d)

#filter_descriptors = filter(lambda d : "OEselma Descriptors" not in d, descriptors)
#descriptors = list(set(descriptors) - set(filter_descriptors))
num_descriptors = len(descriptors)

#---- select only smiles which have these descriptor values
train_smiles = set(train_molecules)
test_smiles = set(test_molecules)

for i,d in enumerate(descriptors):
    avail_rows = df[np.logical_not(df[d].isna())]
    train_smiles = set(avail_rows["SMILES"]).intersection(train_smiles)
    test_smiles = set(avail_rows["SMILES"]).intersection(test_smiles)

train_idxs = np.asarray(sorted([ train_molecules.index(t) for t in train_smiles ]))
test_idxs = np.asarray(sorted([ test_molecules.index(t) for t in test_smiles ]))

train_molecules = np.asarray(train_molecules)
train_molecules = train_molecules[train_idxs]
train_data = train_data[train_idxs]

test_molecules = np.asarray(test_molecules)
test_molecules = test_molecules[test_idxs]
test_data = test_data[test_idxs]
#----
print(descriptors)

[
    'ACD Descriptors;logP',
    'ACD logD-logP;logP',
    'MOE Descriptors;h_ema',
    'MOE Descriptors;h_emd',
    'MOE Descriptors;h_emd_c',
    'MOE Descriptors;h_logp',
    'MOE Descriptors;h_logs',
    'MOE Descriptors;h_log_dbo',
    'MOE Descriptors;h_log_pbo',
    'MOE Descriptors;h_mr',
    'OEselma Descriptors;bromine_count',
    'OEselma Descriptors;carbon_count',
    'OEselma Descriptors;clorine_count',
    'OEselma Descriptors;ertl_tpsa',
    'OEselma Descriptors;fluorine_count',
    'OEselma Descriptors;hba',
    'OEselma Descriptors;hba_lipinski',
    'OEselma Descriptors;hbd',
    'OEselma Descriptors;hbd_lipinski',
    'OEselma Descriptors;halogen_count',
    'OEselma Descriptors;iodine_count',
    'OEselma Descriptors;max_flex_chain_1',
    'OEselma Descriptors;max_flex_chain_2',
    'OEselma Descriptors;max_flex_chain_3',
    'OEselma Descriptors;max_rigid_chain',
    'OEselma Descriptors;mol_volume_2d',
    'OEselma Descriptors;mw',
    'OEselma Descriptors;npsa',
    'OEselma Descriptors;neg_ioniz',
    'OEselma Descriptors;nitrogen_count',
    'OEselma Descriptors;nonpolar_count',
    'OEselma Descriptors;nonpolar_count_per_mw',
    'OEselma Descriptors;rigid_frag_count',
    'OEselma Descriptors;aromatic_ring_count',
    'OEselma Descriptors;atom_count',
    'OEselma Descriptors;bond_count',
    'OEselma Descriptors;rigid_bond_count',
    'OEselma Descriptors;ring_count',
    'OEselma Descriptors;rot_bond_count',
    'OEselma Descriptors;oxygen_count',
    'OEselma Descriptors;psa',
    'OEselma Descriptors;phosphorous_count',
    'OEselma Descriptors;polar_count',
    'OEselma Descriptors;polar_count_per_mw',
    'OEselma Descriptors;pos_ioniz',
    'OEselma Descriptors;silicon_count',
    'OEselma Descriptors;sulphur_count',
    'OEselma Descriptors;tsa',
    'Ertl TPSA;Ertl TPSA',
    'Exact Mass;Exact Mass',
    'Heavy Atom Count;Heavy Atom Count',
    'Molecular Volume (2D);Molecular Volume (2D)',
    'Molecular Weight;Molecular Weight',
    'NPSA;NPSA',
    'PSA;PSA',
    'Ring Count;Ring Count',
    'Rotatable Bond Count;Rotatable Bond Count',
    'AZlogD74;prediction',
    'AZlogD74;sd_prediction',
    'ChromlogD;prediction',
    'ChromlogD;sd_prediction',
    'ALogP',
    'Mol Weight',
    'Molar Refractivity',
    'Polarizability',
    'TPSA',
    'Solubility DD Class;confidence',
    'Solubility DD Class;credibility',
    'SCScore;SCScore',
    'AZlogD74 (NN);nearest_neighbours[]- GT similarity',
    'AZlogD74 (NN);nearest_neighbours[]- GT measured',
    'clogP;clogp',
    'clogP;clogp_error',
    'OEselma Descriptors;part_flex_chain'
]

In [5]:
print(train_data.shape, test_data.shape)

(7205, 9)
(1787, 9)

### Build numpy arrays of descriptors

In [6]:
collect()
train_labels = np.empty((train_data.shape[0], num_descriptors))
test_labels = np.empty((test_data.shape[0], num_descriptors))
train_labels[:] = np.nan
test_labels[:] = np.nan

for i,mol in enumerate(tqdm(train_molecules)):
    data_row = df.loc[df['SMILES'] == mol, descriptors ]
    data_vec = [ r.to_list()[-1] for l,r in data_row.items() ]
    train_labels[i,:] = np.asarray(data_vec)

for i,mol in enumerate(tqdm(test_molecules)):
    data_row = df.loc[df['SMILES'] == mol, descriptors ]
    data_vec = [ r.to_list()[-1] for l,r in data_row.items() ]
    test_labels[i,:] = np.asarray(data_vec)

  0%|          | 0/7205 [00:00<?, ?it/s]

  0%|          | 0/1787 [00:00<?, ?it/s]

In [7]:
print(train_labels.shape, test_labels.shape)

(7205, 74)
(1787, 74)

In [8]:
print(not np.isnan(train_labels).any(), not np.isnan(test_labels).any())

True True

In [9]:
with np.printoptions(2, suppress = True):
    print(choice(train_labels))

[  5.36   5.36   6.66  11.76   9.66  11.84  15.46  -8.3    7.     0.
   0.    31.     2.    52.65   0.     3.     5.     1.     1.     2.
   0.     5.     4.     3.     5.   915.07 552.5  580.78   0.     3.
  28.     0.05   6.     3.    38.    41.    26.     4.    11.     2.
  53.47   0.     5.     0.01   1.     0.     0.   634.    52.65 551.21
  38.   915.07 552.5  580.78  53.47   4.    11.     2.96   0.5    3.9
   0.57   5.46 552.53 155.24  61.17  52.65   0.99   0.84   4.98   0.85
   3.24   4.71   0.     7.  ]

# Setup OLS model

In [10]:
from sklearn import linear_model

#creating a regression object
reg = linear_model.LinearRegression()

#runnin OLS on your data, assuming that you already have arrays x and y
reg.fit( train_data, train_labels )
reg.score(train_data, train_labels)

0.21731159396226127

In [11]:
reg.coef_.shape

(74, 9)

In [12]:
# Using 'auto'/'sum_over_batch_size' reduction type.
mae = tf.keras.losses.MeanAbsoluteError()
mae(test_labels, reg.predict(test_data)).numpy()

2022-01-11 19:26:46.179471: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-11 19:26:46.567211: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6884 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


19.298789978027344

In [13]:
predicted = reg.predict(test_data)

out_dict = dict()
out_dict["Descriptor"] = list()
out_dict["_sigma"] = list()
out_dict["Average"] = list()
out_dict["Test Average"] = test_labels.mean(axis = 0)
out_dict["MAE"] = np.abs(test_labels-predicted).mean(axis = 0)
out_dict["Gauss MAE"] = list()

for i,d in enumerate(descriptors):
    out_dict["Descriptor"].append(d)
    out_dict["Average"].append(df[d].mean(skipna = True)) # population mean for this descriptor
    out_dict["_sigma"].append(df[d].std(skipna = True)) # population mean for this descriptor
    out_dict["Gauss MAE"].append(np.abs(
        test_labels[:,i] - np.random.normal(
            loc = df[d].mean(skipna = True),
            scale = df[d].std(skipna = True),
            size = (test_labels.shape[0],1)
        )
    ).mean())
    
out_dict["MAE/sigma"] = out_dict["MAE"]/out_dict["_sigma"]
out_dict["(GMAE*MAE)/sigma"] = out_dict["MAE/sigma"] * out_dict["Gauss MAE"]
    
#out_dict["abs avg diff"] = np.abs(out_dict["Average"] - out_dict["Test Average"])

pd.set_option("display.max_rows", None, "display.max_columns", None)
out_df = pd.DataFrame(data = out_dict)
print(out_dict["MAE"].sum()/len(descriptors)) # THIS IS WHAT tf.keras.losses.MeanAbsoluteError() DOES
out_df = out_df.sort_values("MAE/sigma")
out_df

19.298790657527803

,Descriptor,_sigma,Average,Test Average,MAE,Gauss MAE,MAE/sigma,(GMAE*MAE)/sigma
45,OEselma Descriptors;silicon_count,0.108439,0.004239,0.000560,0.002336,0.088288,0.021538,0.001902
20,OEselma Descriptors;iodine_count,0.287843,0.028450,0.039731,0.068471,0.262941,0.237878,0.062548
10,OEselma Descriptors;bromine_count,0.265308,0.034856,0.033016,0.067484,0.235247,0.254362,0.059838
62,Mol Weight,301.265463,374.512520,371.636732,102.311704,280.672032,0.339606,95.318041
64,Polarizability,29.516618,37.783778,37.861987,10.196873,27.783745,0.345462,9.598232
63,Molar Refractivity,74.060489,95.559089,95.940680,25.766911,69.454430,0.347917,24.164384
2,MOE Descriptors;h_ema,9.691163,8.864515,8.293012,3.458272,9.000534,0.356848,3.211822
65,TPSA,133.333536,107.810334,104.397905,50.274115,121.795053,0.377055,45.923469
47,OEselma Descriptors;tsa,261.688962,414.138860,413.916620,104.243409,260.178975,0.398349,103.641908
56,Rotatable Bond Count;Rotatable Bond Count,7.213711,6.681677,6.670397,2.915731,7.208879,0.404193,2.913778


In [14]:
latex_dict = dict()
latex_dict["Descriptors"] = out_dict["Descriptor"]
latex_dict["sigma"]       = out_dict["_sigma"]
latex_dict["Avg"]         = out_dict["Average"]
latex_dict["Test avg"]    = out_dict["Test Average"]
latex_dict["MAE"]         = out_dict["MAE"] 
latex_dict["MAE/sigma"]   = out_dict["MAE/sigma"]
latex_df = pd.DataFrame(data = latex_dict)
latex_df = latex_df.sort_values("MAE/sigma")
latex_df

,Descriptors,sigma,Avg,Test avg,MAE,MAE/sigma
45,OEselma Descriptors;silicon_count,0.108439,0.004239,0.000560,0.002336,0.021538
20,OEselma Descriptors;iodine_count,0.287843,0.028450,0.039731,0.068471,0.237878
10,OEselma Descriptors;bromine_count,0.265308,0.034856,0.033016,0.067484,0.254362
62,Mol Weight,301.265463,374.512520,371.636732,102.311704,0.339606
64,Polarizability,29.516618,37.783778,37.861987,10.196873,0.345462
63,Molar Refractivity,74.060489,95.559089,95.940680,25.766911,0.347917
2,MOE Descriptors;h_ema,9.691163,8.864515,8.293012,3.458272,0.356848
65,TPSA,133.333536,107.810334,104.397905,50.274115,0.377055
47,OEselma Descriptors;tsa,261.688962,414.138860,413.916620,104.243409,0.398349
56,Rotatable Bond Count;Rotatable Bond Count,7.213711,6.681677,6.670397,2.915731,0.404193


In [15]:
print(latex_df.to_latex())

\begin{tabular}{llrrrrr}
\toprule
{} &                                        Descriptors &       sigma &         Avg &    Test
avg &         MAE &  MAE/sigma \\
\midrule
45 &                  OEselma Descriptors;silicon\_count &    0.108439 &    0.004239 &    
0.000560 &    0.002336 &   0.021538 \\
20 &                   OEselma Descriptors;iodine\_count &    0.287843 &    0.028450 &    
0.039731 &    0.068471 &   0.237878 \\
10 &                  OEselma Descriptors;bromine\_count &    0.265308 &    0.034856 &    
0.033016 &    0.067484 &   0.254362 \\
62 &                                         Mol Weight &  301.265463 &  374.512520 &  
371.636732 &  102.311704 &   0.339606 \\
64 &                                     Polarizability &   29.516618 &   37.783778 &   
37.861987 &   10.196873 &   0.345462 \\
63 &                                 Molar Refractivity &   74.060489 &   95.559089 &   
95.940680 &   25.766911 &   0.347917 \\
2  &                              MOE Descriptors;h\_ema &    9.691163 &    8.864515 &    
8.293012 &    3.458272 &   0.356848 \\
65 &                                               TPSA &  133.333536 &  107.810334 &  
104.397905 &   50.274115 &   0.377055 \\
47 &                            OEselma Descriptors;tsa &  261.688962 &  414.138860 &  
413.916620 &  104.243409 &   0.398349 \\
56 &          Rotatable Bond Count;Rotatable Bond Count &    7.213711 &    6.681677 &    
6.670397 &    2.915731 &   0.404193 \\
38 &                 OEselma Descriptors;rot\_bond\_count &    7.213711 &    6.681677 &    
6.670397 &    2.915731 &   0.404193 \\
51 &        Molecular Volume (2D);Molecular Volume (2D) &  371.986946 &  592.330629 &  
587.763424 &  155.383329 &   0.417712 \\
25 &                  OEselma Descriptors;mol\_volume\_2d &  371.986946 &  592.330629 &  
587.763424 &  155.383329 &   0.417712 \\
6  &                             MOE Descriptors;h\_logs &    5.975325 &    9.838051 &    
9.752189 &    2.600139 &   0.435146 \\
34 &                     OEselma Descriptors;atom\_count &   15.766603 &   25.429675 &   
25.508674 &    6.920609 &   0.438941 \\
50 &                  Heavy Atom Count;Heavy Atom Count &   15.757548 &   25.426522 &   
25.506995 &    6.919384 &   0.439116 \\
26 &                             OEselma Descriptors;mw &  227.821937 &  373.328639 &  
371.636766 &  102.316792 &   0.449109 \\
52 &                  Molecular Weight;Molecular Weight &  227.821937 &  373.328639 &  
371.636766 &  102.316792 &   0.449109 \\
49 &                              Exact Mass;Exact Mass &  227.548842 &  372.828687 &  
371.268124 &  102.207349 &   0.449167 \\
35 &                     OEselma Descriptors;bond\_count &   16.980448 &   26.935092 &   
27.180750 &    7.750208 &   0.456420 \\
53 &                                          NPSA;NPSA &  200.207940 &  307.642466 &  
311.829856 &   91.812098 &   0.458584 \\
27 &                           OEselma Descriptors;npsa &  200.207940 &  307.642466 &  
311.829856 &   91.812098 &   0.458584 \\
41 &              OEselma Descriptors;phosphorous\_count &    0.474829 &    0.122751 &    
0.123111 &    0.221552 &   0.466593 \\
3  &                              MOE Descriptors;h\_emd &   10.821464 &   12.580101 &   
12.460378 &    5.054848 &   0.467113 \\
11 &                   OEselma Descriptors;carbon\_count &   11.377305 &   17.636929 &   
17.901511 &    5.320611 &   0.467651 \\
4  &                            MOE Descriptors;h\_emd\_c &    5.020204 &    6.492526 &    
6.514657 &    2.400257 &   0.478119 \\
54 &                                            PSA;PSA &  101.197251 &  106.494461 &  
102.077127 &   50.681520 &   0.500819 \\
40 &                            OEselma Descriptors;psa &  101.197251 &  106.494461 &  
102.077127 &   50.681520 &   0.500819 \\
73 &                OEselma Descriptors;part\_flex\_chain &    4.183248 &    4.829146 &    
4.982652 &    2.105559 &   0.503331 \\
48 &                                Ertl TPSA;Ertl TPSA &   94.357347 